<a href="https://colab.research.google.com/github/MrWhoCoded/MAD_project/blob/main/FaceNet_nontuned.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from keras_facenet import FaceNet
import cv2
import os
import dlib
from google.colab.patches import cv2_imshow
from scipy.spatial.distance import cosine

In [ ]:
!pip install keras_facenet

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 67.8 MB/s eta 0:00:00
  Created wheel for keras_facenet: filename=keras_facenet-0.3.2-py3-none-any.whl size=10367 sha256=6caf2f685792b655505554e0dec5b815f994cbeb5bb15d5f1703fb8a5feb0677
  Stored in directory: /root/.cache/pip/wheels/05/b0/f5/19ac49fedc10b1df3ee56b096edbcfa39d45794fccc6bcdbbf
Successfully built keras_facenet


In [ ]:
embeddings = []
labels = []

for image in os.listdir(r"/content/drive/MyDrive/Football players"):
    print(f"processing {image}")
    img = cv2.imread(os.path.join(r"/content/drive/MyDrive/Football players", image))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    model = FaceNet()
    detector = dlib.get_frontal_face_detector()
    faces = detector(img)

    # Process each detected face
    for face in faces:
        x1 = face.left()
        y1 = face.top()
        x2 = face.right()
        y2 = face.bottom()

        face_frame = img[y1:y2, x1:x2]
        face_frame = cv2.resize(face_frame, (160, 160))

        # Add an extra dimension to the face_frame to simulate a batch of one image
        face_frame = np.expand_dims(face_frame, axis=0)

        model_embedding = model.embeddings(face_frame)
        embeddings.append(model_embedding)
        labels.append(image.split('.')[0])

embeddings = np.array(embeddings)
labels = np.array(labels)

np.savez_compressed(r"/content/drive/MyDrive/Football players/face_embeddings.npz", embeddings=embeddings, labels=labels)
print("Face embeddings and labels saved to face_embeddings.npz")

    #cv2_imshow(img)
    #cv2.waitKey(0)
    #cv2.destroyAllWindows()

processing Neymar1.jpg


1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
processing Messi1.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
processing Ronaldo1.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
processing Ronaldo2.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
processing Messi2.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
processing Mbape1.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step
processing Mbape2.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
processing Neymar2.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
processing Haland1.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
processing Haland2.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
Face embeddings and labels saved to face_embeddings.npz


In [ ]:
def compareFaces(input_image, embedding_file):
    img = input_image

    model = FaceNet()

    detector = dlib.get_frontal_face_detector()
    faces = detector(img)

    input_embeddings = []

    for face in faces:
        x1 = face.left()
        y1 = face.top()
        x2 = face.right()
        y2 = face.bottom()

        face_frame = img[y1:y2, x1:x2]
        face_frame = cv2.resize(face_frame, (160, 160))

        face_frame = np.expand_dims(face_frame, axis=0)

        model_embedding = model.embeddings(face_frame)
        input_embeddings.append(model_embedding)


    input_embeddings = np.array(input_embeddings)

    data = np.load(embedding_file)
    embeddings = data['embeddings']
    labels = data['labels']

    for input_embedding in input_embeddings:
        min_distance = float("inf")
        best_label = None

        for label, embedding in zip(labels, embeddings):
            distance = cosine(input_embedding.flatten(), embedding.flatten())
            print(f"Distance between input face {label}: {distance}")
            if distance < min_distance:
                min_distance = distance
                best_label = label

        print(f"\nClosest match for input face {best_label} (distance = {min_distance})\n")

In [ ]:
input_image_path = r"/content/drive/MyDrive/Football players/Test images/Messi_test.jpg"
embedding_file = r"/content/drive/MyDrive/Football players/face_embeddings.npz"

# Load the image using cv2.imread()
input_image = cv2.imread(input_image_path)

# Check if the image was loaded successfully
if input_image is None:
    print(f"Error: Could not load input image from {input_image_path}")
else:
    # Convert the image to RGB color space before passing it to compareFaces
    input_image_rgb = cv2.cvtColor(input_image, cv2.COLOR_BGR2RGB)
    compareFaces(input_image_rgb, embedding_file)

1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
Distance between input face Neymar1: 0.9654976725578308
Distance between input face Messi1: 0.23404079675674438
Distance between input face Ronaldo1: 1.0155649185180664
Distance between input face Ronaldo2: 1.0469441413879395
Distance between input face Messi2: 0.3765149712562561
Distance between input face Mbape1: 1.3023087978363037
Distance between input face Mbape2: 1.1917893886566162
Distance between input face Neymar2: 0.9263049364089966
Distance between input face Haland1: 0.9796553254127502
Distance between input face Haland2: 0.9173915386199951

Closest match for input face Messi1 (distance = 0.23404079675674438)

